#### The imports

In [1]:
from itertools import chain
from collections import defaultdict
import time

#### Utils

In [2]:
def lists_to_binarylists(integer_lists):
    max_num = max(chain(*integer_lists))
    binary_lists = []
    for itemset in integer_lists:
        binary = [0] * max_num
        for num in itemset:
            binary[num-1] = 1
        binary_lists.append(binary)
    return (binary_lists, max_num)

In [3]:
def postprocess(freq_itemsets, max_num):
    result = {}
    for k, v in freq_itemsets.items():
        output_dict = {}
        for key, value in v.items():
            if value not in output_dict:
                output_dict[value] = []
            binary_list = get_binary_list(key, max_num)
            indices = [i+1 for i, x in enumerate(binary_list) if x == 1]
            output_dict[value].append(indices)
        if len(output_dict) > 0:
            result[k] = output_dict
    return result

#### Algorithm

In [4]:
def get_binary_list(frozenset_number, n):
    number = int(list(frozenset_number)[0])
    binary_string = bin(number)[2:]  # remove '0b' prefix
    binary_list = [int(bit) for bit in binary_string]
    binary_list = [0] *(n - len(binary_list)) + binary_list
    return binary_list

In [5]:
def get_number(binary_list):
    binary_string = ''.join(str(bit) for bit in binary_list)  # convert to string
    number = int(binary_string, 2)
    return number

In [6]:
def frequent_mining(key, k, n, transactions, min_support, output_freq_itemsets, output_freq_itemsets_occurrences):
    if k > n:
        return
    
    binary_list = get_binary_list(key, n)
    last_index_of_1 = n - 1 - list(reversed(binary_list)).index(1)
    if last_index_of_1 >= n:
        return
    
    itemset_counts = defaultdict(int)
    itemset_occurrences = defaultdict(int)
    for i in range(last_index_of_1 +1, n):
        template = list(binary_list)
        template[i] = 1
        decimal_number = get_number(template)
        
        masked_lists = [[a & b for a, b in zip(itemset, template)] for itemset in transactions]
        itemset_counts[decimal_number] = masked_lists.count(template)
        itemset_occurrences[decimal_number] = (masked_lists.count(template), [i for i, x in enumerate(masked_lists) if x == template])
        
        new_key = frozenset([decimal_number])
        frequent_mining(new_key, k +1, n, transactions, min_support, output_freq_itemsets, output_freq_itemsets_occurrences)
        
    if output_freq_itemsets.get(k, None) is not None:
        output_freq_itemsets[k].update({frozenset([item]): count for item, count in itemset_counts.items() if count >= min_support})
        output_freq_itemsets_occurrences[k].update({frozenset([item]): occ_list for item, (count, occ_list) in itemset_occurrences.items() if count >= min_support})
    else:
        output_freq_itemsets[k] = {frozenset([item]): count for item, count in itemset_counts.items() if count >= min_support}
        output_freq_itemsets_occurrences[k] = {frozenset([item]): occ_list for item, (count, occ_list) in itemset_occurrences.items() if count >= min_support}
    

In [7]:
def frequent_itemsets(transactions, min_support):
    #print("Start frequent mining process...")
    itemset_counts = defaultdict(int)
    itemset_occurrences = defaultdict(int)
    #print("Obtaining the 1-frequent items...")
    n = len(transactions[0])
    for i in range(n):
        template = [0] *n
        template[i] = 1
        decimal_number = get_number(template)
        
        masked_lists = [[a & b for a, b in zip(itemset, template)] for itemset in transactions]
        itemset_counts[decimal_number] = masked_lists.count(template)
        itemset_occurrences[decimal_number] = (masked_lists.count(template), [i for i, x in enumerate(masked_lists) if x == template])
    #print("1-frequent items done")

    freq_itemsets = dict()
    freq_itemsets_occurrences = dict()
    freq_itemsets[1] = {frozenset([item]): count for item, count in itemset_counts.items() if count >= min_support}
    freq_itemsets_occurrences[1] = {frozenset([item]): occ_list for item, (count, occ_list) in itemset_occurrences.items() if count >= min_support}
    #print(freq_itemsets)

    for key, item in freq_itemsets[1].items():
        frequent_mining(key, 2, n, transactions, min_support, freq_itemsets, freq_itemsets_occurrences)
    
    #print("Done...")
    return (freq_itemsets, freq_itemsets_occurrences)

#### Example usage

In [8]:
dataset = [[1, 2, 3], [2, 3, 4], [1, 2, 4], [1, 3, 4], [2, 3]]
min_support = 1

start_time = time.time()
(bDataset, max_num) = lists_to_binarylists(dataset)
end_time = time.time()
execution_time = end_time - start_time
print("Preprocess execution time:", execution_time, "seconds")
print("Settings:")
print("\ttransactions:", bDataset)
print("\tthreshold:", min_support)

start_time = time.time()
(freq_itemsets, freq_itemsets_occurrences) = frequent_itemsets(bDataset, min_support)
end_time = time.time()
execution_time = end_time - start_time
print("Mining execution time:", execution_time, "seconds")

print("Output")
output = postprocess(freq_itemsets, max_num)
for k, v in output.items():
    print(k, v)
#print("Output", freq_itemsets)
#print("Output occurrences", freq_itemsets_occurrences)


Preprocess execution time: 0.0 seconds
Settings:
	transactions: [[1, 1, 1, 0], [0, 1, 1, 1], [1, 1, 0, 1], [1, 0, 1, 1], [0, 1, 1, 0]]
	threshold: 1
Mining execution time: 0.0 seconds
Output
1 {3: [[1], [4]], 4: [[2], [3]]}
3 {1: [[1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4]]}
2 {2: [[1, 2], [1, 3], [1, 4], [2, 4], [3, 4]], 3: [[2, 3]]}
